In [1]:
# autoreload
%load_ext autoreload
%autoreload 2


In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from copy import deepcopy
from graphs.llama_graph import TransformerEncoderGraph


In [4]:
# this is a model
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
# not used yet
# tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
state_dict = model.state_dict()

print(model)

# defaults merge the minimum so we need to add flags or change defaults
# example = qk=True, merge_type='all', classifier=??
graph1 = TransformerEncoderGraph(deepcopy(model)).graphify()

LlamaModel(
  (embed_tokens): Embedding(49152, 576, padding_idx=2)
  (layers): ModuleList(
    (0-29): 30 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=576, out_features=576, bias=False)
        (k_proj): Linear(in_features=576, out_features=192, bias=False)
        (v_proj): Linear(in_features=576, out_features=192, bias=False)
        (o_proj): Linear(in_features=576, out_features=576, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
        (up_proj): Linear(in_features=576, out_features=1536, bias=False)
        (down_proj): Linear(in_features=1536, out_features=576, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((576,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding

TypeError: TransformerEncoderGraph.__init__() missing 1 required positional argument: 'modules'

In [12]:
params = []
for name,param in model.named_parameters():
    tmp_name = name
    if "layers." in name:
        # find the index of the second '.'
        second_dot_index = name.find('.', name.find('.') + 1)
        tmp_name = name[second_dot_index+1:]
    params.append(tmp_name)

unique_params = set(params)
print(unique_params)



{'mlp.up_proj.weight', 'self_attn.v_proj.weight', 'mlp.gate_proj.weight', 'embed_tokens.weight', 'self_attn.q_proj.weight', 'norm.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight', 'self_attn.o_proj.weight', 'mlp.down_proj.weight', 'self_attn.k_proj.weight'}


In [13]:
for param in unique_params:
    print(param)

mlp.up_proj.weight
self_attn.v_proj.weight
mlp.gate_proj.weight
embed_tokens.weight
self_attn.q_proj.weight
norm.weight
input_layernorm.weight
post_attention_layernorm.weight
self_attn.o_proj.weight
mlp.down_proj.weight
self_attn.k_proj.weight


In [14]:
for module in model.modules():
    print(module)

LlamaModel(
  (embed_tokens): Embedding(49152, 576, padding_idx=2)
  (layers): ModuleList(
    (0-29): 30 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=576, out_features=576, bias=False)
        (k_proj): Linear(in_features=576, out_features=192, bias=False)
        (v_proj): Linear(in_features=576, out_features=192, bias=False)
        (o_proj): Linear(in_features=576, out_features=576, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
        (up_proj): Linear(in_features=576, out_features=1536, bias=False)
        (down_proj): Linear(in_features=1536, out_features=576, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((576,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding

In [16]:
dir(module)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_dynamic_frequency_update',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_is_hf_initialized',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modu

In [24]:
for name, tmp_module in module.named_modules():
    print(name)
    print(tmp_module)



LlamaRotaryEmbedding()


In [27]:
for x in module.modules():
    print(x)
    break

LlamaRotaryEmbedding()


In [28]:
graph1 = TransformerEncoderGraph(deepcopy(model), modules={}).graphify()

KeyError: 'emb'

In [30]:
for name, module in model.named_modules():
    print(name)


embed_tokens
layers
layers.0
layers.0.self_attn
layers.0.self_attn.q_proj
layers.0.self_attn.k_proj
layers.0.self_attn.v_proj
layers.0.self_attn.o_proj
layers.0.self_attn.rotary_emb
layers.0.mlp
layers.0.mlp.gate_proj
layers.0.mlp.up_proj
layers.0.mlp.down_proj
layers.0.mlp.act_fn
layers.0.input_layernorm
layers.0.post_attention_layernorm
layers.1
layers.1.self_attn
layers.1.self_attn.q_proj
layers.1.self_attn.k_proj
layers.1.self_attn.v_proj
layers.1.self_attn.o_proj
layers.1.self_attn.rotary_emb
layers.1.mlp
layers.1.mlp.gate_proj
layers.1.mlp.up_proj
layers.1.mlp.down_proj
layers.1.mlp.act_fn
layers.1.input_layernorm
layers.1.post_attention_layernorm
layers.2
layers.2.self_attn
layers.2.self_attn.q_proj
layers.2.self_attn.k_proj
layers.2.self_attn.v_proj
layers.2.self_attn.o_proj
layers.2.self_attn.rotary_emb
layers.2.mlp
layers.2.mlp.gate_proj
layers.2.mlp.up_proj
layers.2.mlp.down_proj
layers.2.mlp.act_fn
layers.2.input_layernorm
layers.2.post_attention_layernorm
layers.3
layers.

In [31]:
module.named_modules()["layers.29.mlp.up_proj"]

TypeError: 'generator' object is not subscriptable

In [32]:
named_modules_dict = dict(model.named_modules())

In [34]:
dir(named_modules_dict["layers.29.mlp.up_proj"])

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_is_hf_initialized',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_nam

In [38]:
for x in named_modules_dict["layers.29.mlp.up_proj"].parameters():
    print(x.shape)
    


torch.Size([1536, 576])
